In [1]:
###setup

import urllib.request
import json 
import datetime
import pandas as pd
import numpy as np
import array
import matplotlib
import seaborn


In [5]:
print(datetime.datetime.now().time())

###get human gene GO term associations
#curl -X GET --header 'Accept: application/json' 'https://api.monarchinitiative.org/api/mart/gene/phenotype/NCBITaxon:9606'
GO_data = []
with urllib.request.urlopen("https://api.monarchinitiative.org/api/mart/gene/function/NCBITaxon:9606") as url:
    GO_data.append(json.loads(url.read().decode()))

print(datetime.datetime.now().time())

16:01:22.767628
16:01:37.276831


In [6]:
print(len(GO_data[0]))
print(GO_data[0][1])

21480
{'objects': ['GO:0006357', 'GO:0006359', 'GO:0005634', 'GO:0001228', 'GO:0043565', 'GO:0006366', 'GO:0045944', 'GO:0046872'], 'subject': 'UniProtKB:P36508', 'subject_label': 'ZNF76', 'relation': None}


In [7]:
###Create gene - multi-term dictionary

print(datetime.datetime.now().time())

go_annot = dict()
for i in range(1,len(GO_data[0])):
    if i % 100 == 0 :
        print(str(i)+"\t"+GO_data[0][i]['subject'])
        
    go_annot[GO_data[0][i]['subject']] = GO_data[0][i]['objects'] 
                 
keys = go_annot.keys()
print(len(keys))

print(datetime.datetime.now().time())


16:02:14.710402
100	UniProtKB:Q9H2S6
200	UniProtKB:O75715
300	UniProtKB:P05109
400	UniProtKB:Q14139
500	UniProtKB:O94851
600	UniProtKB:P13866
700	UniProtKB:Q9Y3I0
800	UniProtKB:O00287
900	UniProtKB:Q8NC44
1000	UniProtKB:Q9UBP4
1100	UniProtKB:P84095
1200	UniProtKB:Q9H832
1300	UniProtKB:Q9Y4Y9
1400	UniProtKB:Q8NC42
1500	UniProtKB:Q53GL7
1600	UniProtKB:A6NLU5
1700	UniProtKB:A0A096LNP1
1800	UniProtKB:O60894
1900	UniProtKB:Q9NYD6
2000	UniProtKB:Q6NSI4
2100	UniProtKB:P56962
2200	UniProtKB:Q9Y2K9
2300	UniProtKB:Q9UN86
2400	UniProtKB:Q86WA9
2500	UniProtKB:Q8NFW5
2600	UniProtKB:Q8N1Q8
2700	UniProtKB:Q9BQS6
2800	UniProtKB:Q14244
2900	UniProtKB:P23280
3000	UniProtKB:Q07343
3100	UniProtKB:Q15475
3200	UniProtKB:Q6P499
3300	UniProtKB:O75344
3400	UniProtKB:Q9UNX9
3500	UniProtKB:Q16718
3600	UniProtKB:Q4KMG9
3700	UniProtKB:Q9HBF4
3800	UniProtKB:Q5HY92
3900	UniProtKB:Q96I24
4000	UniProtKB:Q5XKL5
4100	UniProtKB:Q9Y5P8
4200	UniProtKB:A8MTL0
4300	UniProtKB:Q14676
4400	UniProtKB:Q9C026
4500	UniProtKB:P40925

In [8]:
###Build dataframe with GO term presence/absence for all human genes
###Currently something is very SLOW
###Probably too many .index() calls?

print(datetime.datetime.now().time())

print(go_annot['UniProtKB:P13671'])

###count dictionary for all GO terms with human gene coverage
go_total = dict()
for k in keys:
    curgo = go_annot[k]
    for g in curgo:
        if(g not in go_total) :
            go_total[g] = 1
        else :
            val = go_total[g]
            go_total[g] = val+1
    
go_annot_ref_list = list(go_total.keys())

print(datetime.datetime.now().time())


16:02:19.664252
['GO:0045766', 'GO:0006958', 'GO:0001701', 'GO:0001970', 'GO:0030449', 'GO:0045087', 'GO:0006956', 'GO:0005579', 'GO:0005576', 'GO:0005515', 'GO:0070062', 'GO:0019835']
16:02:19.761041


In [9]:
###column index lookup dict
print(datetime.datetime.now().time())

go_annot_ref_index = dict()

for i in range(0, len(go_annot_ref_list)):
    go_annot_ref_index[go_annot_ref_list[i]] = i
               
print(datetime.datetime.now().time())

16:02:23.030204
16:02:23.034928


In [10]:
print(datetime.datetime.now().time())

#go_annot_df = pd.DataFrame(index=np.arange(0, len(keys)),columns=go_annot_list)

go_annot_mat = np.zeros(shape = (len(keys), len(go_annot_ref_list)))

count = 0
lengo = len(go_annot_ref_list)
for k in keys:
    curgo = go_annot[k]
    for g in curgo:
        go_annot_mat[count][go_annot_ref_index[g]] = 1
    #print(len(curgo))
    if count % 1000 == 0:
        print (count)
    count = count+1

go_annot_df = pd.DataFrame(go_annot_mat,columns=go_annot_ref_list, index=keys)

print(go_annot_df.shape)

print(datetime.datetime.now().time())

16:02:25.597254
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
(19864, 16828)
16:02:26.047267


In [11]:
###output full annotation matrix to file, for the record
print(datetime.datetime.now().time())
#np.savetxt(r'./human_GO_annotation_matrix.txt', go_annot_df.values, fmt='%d')
go_annot_df.to_csv(r'human_GO_annotation_matrix.txt', sep='\t')
print(datetime.datetime.now().time())


16:02:28.147218
16:07:08.795863


In [12]:
###GO annotation column sums, for enrichment analysis bg etc.
go_col_sum = go_annot_df[go_annot_ref_list].sum()

print(go_col_sum)

GO:0006357     386.0
GO:0006359      12.0
GO:0005634    4821.0
GO:0001228     115.0
GO:0043565     349.0
GO:0006366     517.0
GO:0045944     939.0
GO:0046872    2176.0
GO:0009611      59.0
GO:0005654    2722.0
GO:0006351    1694.0
GO:0001525     196.0
GO:0005667     163.0
GO:0060743       3.0
GO:0040008      45.0
GO:0008544      78.0
GO:0006355    1274.0
GO:0060527       5.0
GO:0033574      32.0
GO:0005783     806.0
GO:0048193       9.0
GO:0030008       7.0
GO:0005794     787.0
GO:0045836       5.0
GO:0051260     185.0
GO:0007338      51.0
GO:0006338      73.0
GO:0045740      39.0
GO:0000789       1.0
GO:0009790      35.0
               ...  
GO:0009628       1.0
GO:0016590       1.0
GO:1904045       1.0
GO:0032831       1.0
GO:0002362       1.0
GO:0002851       1.0
GO:0048613       1.0
GO:0060446       1.0
GO:0060461       1.0
GO:0014822       1.0
GO:0048371       1.0
GO:0004102       1.0
GO:0032401       1.0
GO:0044268       1.0
GO:0019319       1.0
GO:0019249       1.0
GO:0045819   

In [13]:
print(go_annot_df.shape)
#go_annot_df_filter = go_annot_df.loc[(go_annot_df!=0).any(0)]
#go_annot_df_filter = go_annot_df.loc[(go_col_sum>1).any(0)]
go_annot_df_filter = go_annot_df.drop([col for col, val in go_annot_df.sum().iteritems() if val < 2], axis=1, inplace=False)
#go_col_sum
print(go_annot_df_filter.shape)

(19864, 16828)
(19864, 11637)


In [14]:
###output filtered annotation matrix to file, for the record
print(datetime.datetime.now().time())
#np.savetxt(r'./human_GO_annotation_matrix.txt', go_annot_df.values, fmt='%d')
go_annot_df_filter.to_csv(r'human_GO_annotation_matrix_great1.txt', sep='\t')
print(datetime.datetime.now().time())

16:07:36.813652
16:10:25.589641


In [15]:
###creata clustered heatmap visualization
print(datetime.datetime.now().time())
g = seaborn.clustermap(go_annot_df_filter, metric="Hamming", linkage="single")
print(datetime.datetime.now().time())

16:11:09.365562


/Users/marcin/anaconda/lib/python3.6/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The axisbg attribute was deprecated in version 2.0. Use facecolor instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


RecursionError: maximum recursion depth exceeded while calling a Python object